In [ ]:
import numpy as np
from astropy.wcs import WCS
import glue_jupyter as gj
from glue.core import Data
from glue.plugins.wcs_autolinking.wcs_autolinking import WCSLink
from specutils import Spectrum1D
from glue.core.link_helpers import LinkSame

In [ ]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['WAVE']
wcs1.wcs.crval = [10]
wcs1.wcs.cdelt = [3]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['cm']

d1 = Data(temp1=np.random.random(10))
d1.get_component('temp1').units = 'deg_C'
d1.coords = wcs1

In [ ]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['WAVE']
wcs1.wcs.crval = [10]
wcs1.wcs.cdelt = [3]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['cm']

d2 = Data(temp2=np.random.random(10) + 273)
d2.get_component('temp2').units = 'K'
d2.coords = wcs1

In [ ]:
from astropy import units as u
from glue.core.units import unit_converter

@unit_converter('custom-astropy')
class UnitConverterWithTemperature:

    def equivalent_units(self, data, cid, units):
        equivalencies = u.temperature() if 'temp' in cid.label.lower() else None
        return map(u.Unit(units).find_equivalent_units(include_prefix_units=True, equivalencies=equivalencies), str)

    def to_unit(self, data, cid, values, original_units, target_units):
        equivalencies = u.temperature() if 'temp' in cid.label.lower() else None
        return (values * u.Unit(original_units)).to_value(target_units, equivalencies=equivalencies)

In [ ]:
from glue.config import settings
settings.UNIT_CONVERTER = 'custom-astropy'

In [ ]:
from jdaviz import Specviz
specviz = Specviz()
specviz.app

In [ ]:
specviz.app.add_data(d1, "d1")
specviz.app.add_data(d2, "d2")
specviz.app.data_collection.add_link(WCSLink(d1, d2))


### The following cell changes the display of the flux unit

In [ ]:
# Y conversion works but X does not allow conversion from wavelength to frequency

# specviz.app.get_viewer("spectrum-viewer").state.x_display_unit = "MHz"
specviz.app.get_viewer("spectrum-viewer").state.y_display_unit = "K"

In [ ]:
# Shows data retains units while being displayed in different units
viewer = specviz.app.get_viewer("spectrum-viewer")
viewer.data()